# Exercise 4 -Computer Vision


### 4.1 - Face Detection and Tracking
In this task you will implement face detection and tracking using OpenCV. Specifically we are utilizing Cascade classifiers which implements Viola-Jones detection algorithm.

**Reference**
- [OpenCV documentation on cascade classifier](https://docs.opencv.org/master/db/d28/tutorial_cascade_classifier.html)

### 4.1.1
Execute the code below to initiate the cascadee classifier and the utility libraries

In [2]:
import cv2
import time
import numpy as np
import time
import imutils

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') 

### 4.1.2

Similar to Task 3, the first step is to obtain a frame from video file and pre-processing it. 

**Your task**

Complete prep() function below which performs following using opencv and imutils libraries. The steps already implemented are marked with a tick "✓"

- [x] Takes a frame from video feed as the input
- [ ] Resize the frame while protecting the aspect ratio (width = 600) 
- [ ] Flip the image
- [ ] Convert the frame to grayscale image
- [x] Return grayscale image and resized image 

**References**

- [imutils documentation](https://github.com/PyImageSearch/imutils#imutils)
- [Fip an array with OpenCV](https://docs.opencv.org/4.x/d2/de8/group__core__array.html#gaca7be533e3dac7feb70fc60635adf441)
- [color conversion with OpenCV](https://docs.opencv.org/4.x/d8/d01/group__imgproc__color__conversions.html)

In [2]:
def prep(img):
    ## ToDo 4.1.2
    #  1. resize using  imutils.resize()
    #  2. flip image using cv2.flip()
    #  3. convert to gray color using cv2.cvtColor()
    img = imutils.resize(img,width=600)
    #img_flipped = cv2.flip(img,0)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    return gray, img    

# 4.1.3

In 4.1.1 we initialized an instance of cascade classifier. Tracking a face can be broken down into 3 steps as below

1. Detect Faces and ROIs

   The cascade classifier has a member function which can detect faces of multiple scales in a given image. The area where a face is detected becomes a region of interest (ROI) for extracting meaningful information. 

    **References** : 
    [Multiscale face detection member function of cascade classifier](https://docs.opencv.org/3.4/d1/de5/classcv_1_1CascadeClassifier.html#a90fe1b7778bed4a27aa8482e1eecc116)


2. Extract trackable features 

    Shi-Tomasi Corner Detector is an implementation in openCV which extracts information from the ROI input. The extracted information are points on the face which are are trackable across a sequence of moving frames (a video).

    **References** : 
    [OpenCV Trackable feature extraction function(Shi-Tomasi Corner Detector)](https://docs.opencv.org/4.5.2/d4/d8c/tutorial_py_shi_tomasi.html)


3. Calculate the optical flow

    These trackable points are used to calculate the optical flow of the faces with calcOpticalFlowPyrLK() function. The tracking is visualized via OpenCV drawing tools.

    **References** : 
    - [Optical Flow calculation](https://docs.opencv.org/4.5.3/d4/dee/tutorial_optical_flow.html)
    - [OpenCV drawing functions](https://docs.opencv.org/4.5.2/dc/da5/tutorial_py_drawing_functions.html)

**Your task**

Complete the function which perfoms following

- [x] Takes grayscale image and resized image as the input
- [x] Detect faces in graycale image using cascade classifier. detectMultiscale() function returns detected faces as rectangles ( Top left x coordinate, Top left y coordinate, width, height)
- [ ] Draw a rectangle around detected faces using OpenCV drawing functions
- [ ] Slice a region of interest (ROI) from grayecale image corresponding to the detections
- [x] Extract good features to track (p0), from OpenCV goodFeaturesToTrack() function.
- [ ] Convert the array p0 from current format [[[x1,y1],[x2,y2],....]] to --> [[x1,y1],[x2,y2],....]. Tip : print p0 to observe current format
- [ ] The points are located with respect to the ROI coordinates. Convert them to image coordinates

In [3]:
def get_trackable_points(gray,img):
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)
    p0 = []
    if len(faces) != 0:
        
        rois = []
        ## ToDO 4.1.3
        for (x,y,w,h) in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
            roi_gray = gray[y:y+h,x:x+w]  
            p0tmp = cv2.goodFeaturesToTrack(roi_gray,maxCorners=70,qualityLevel=0.001,minDistance=5)
            p0tmp = p0tmp.reshape(-1,2)
            p0tmp = [[x+corner[0],y+corner[1]] for corner in p0tmp]
            rois.extend(p0tmp)
        
        p0 = rois
        # ToDO 4.1.3
        #  covert fromat of p0 to [[x1,y1],[x2,y2],....] 
        #  convert points from ROI to image coordinates
   
    p0 = np.array(p0,dtype=np.float32)

    return p0, faces, img

**Your task**

Complete the do_track_face() function which perfoms following

- [x] Usecv2.calcOpticalFlowPyrLK()to calculate the optical flow for tracking face
- [ ] Select the valid points from p1. Note that  isFound == 1 for valid points 
- [ ] Return the valid points as a numpy array

In [4]:
def do_track_face(gray_prev, gray, p0):
    p1, isFound, err = cv2.calcOpticalFlowPyrLK(gray_prev, gray, p0, 
                                                            None,
                                                            winSize=(31,31),
                                                            maxLevel=10,
                                                            criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.03),
                                                            flags=cv2.OPTFLOW_LK_GET_MIN_EIGENVALS,
                                                            minEigThreshold=0.00025)
    ## ToDo 4.1.3 - Select valid points from p1
    # Select valid points from p0   
    # return a numpy array of selected points from p0
    valid = p1[(isFound ==1).ravel()]
    #valid = p1[isFound ==1]
    return valid

### 4.1.4

Run the program to view the final output of face tracking. Remember to enter the correct path to video file provided.

In [5]:
frame_rate = 30
prev = 0
gray_prev = None
p0 = []
cam = cv2.VideoCapture("Face.mp4")
#cam = cv2.VideoCapture(0)

if not cam.isOpened():
    raise Exception("Could not open camera/file")
    
while True:
    time_elapsed = time.time() - prev
    
    if time_elapsed > 1./frame_rate:
        
        ret_val,img = cam.read()
        
        if not ret_val:
                cam.set(cv2.CAP_PROP_POS_FRAMES, 0)  # restart viiideo
                gray_prev = None  # previous frame
                p0 = []  # previous points
                continue
        prev = time.time()
        
        gray, img = prep(img)

        if len(p0) <= 10: 
            p0, faces, img = get_trackable_points(gray,img)
        
        else:
            for (x,y,w,h) in faces:
                cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
                p1 = do_track_face(gray_prev, gray, p0)
            for i in p1:
                cv2.drawMarker(img, (round(i[0]), round(i[1])),[255,0,0],0)
            p0 = p1
        
        gray_prev = gray.copy()
        cv2.imshow('Video feed', img)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break
              
cv2.destroyAllWindows()
cam.release()

In [1]:
# 5

import sys
import rclpy
from rclpy.node import Node
from cv_bridge import CvBridge
import numpy as np
import cv2
from sensor_msgs.msg import Image, CameraInfo
#print(sys.path)

bridge = CvBridge()

class Get_Images(Node):

    def __init__(self):
        # Initialize the node
        super().__init__('Image_Subscriber')
        # Initialize the subscriber
        self.subscription_ = self.create_subscription( Image,'/image_raw', self.listener_callback,10)
        self.subscription_  # prevent unused variable warning
        timer_period = 0.1  # seconds
        self.timer = self.create_timer(timer_period, self.timer_callback)
        self.p0 = []
        self.frame_counter = 0
        self.frame = None

    def listener_callback(self, msg):
        height = msg.height
        width = msg.width
        channel = msg.step//msg.width
        #frame = np.reshape(msg.data, (height, width, channel))
        self.frame =  bridge.imgmsg_to_cv2(msg, desired_encoding='bgr8')
        return self.frame
    
    def timer_callback(self):
        if self.frame is None:
            return
        gray, img = prep(self.frame)
        self.frame_counter += 1
        

        #if len(self.p0) <= 10 or self.frame_counter % 60 == 0: 
        p0, faces, img = get_trackable_points(gray,img)
        self.p0 = p0
        gray_prev = gray.copy()
        self.frame_counter = 0
        #else:
        for (x,y,w,h) in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
            p1 = do_track_face(gray_prev, gray, self.p0)
            for i in p1:
                cv2.drawMarker(img, (round(i[0]), round(i[1])),[255,0,0],0)
            self.p0 = p1


        gray_prev = gray.copy()
        cv2.imshow('Video feed', img)
        key = cv2.waitKey(1) & 0xFF

                
    def stop_stream(self):
        self.get_logger().info('Stopping the stream ...')
        

try:
    rclpy.init(args=None)
    image_subscriber = Get_Images() 
    rclpy.spin(image_subscriber)

except KeyboardInterrupt:
    # executes on keyboard kernal interrupt with double pressing button "i"
    image_subscriber.stop_stream()
    image_subscriber.destroy_node()
    rclpy.shutdown()

NameError: name 'prep' is not defined